In [ ]:
import cv2
import numpy as np
import os
from tqdm import tqdm
from skimage.feature import local_binary_pattern
import pandas as pd

In [ ]:
AMBON_DIR = 'ambon'
BARANGAN_DIR = 'barangan'
CAVENDISH_DIR = 'cavendish'
GENDERUWO_DIR = 'genderuwo'
JAVAICECREAM_DIR = 'javaicecream'
KEPOK_DIR = 'kepok'
MAS_DIR = 'mas'
NANGKA_DIR = 'nangka'
RAJA_DIR = 'raja'
SUSU_DIR = 'susu'
TANDUK_DIR = 'tanduk'
ULI_DIR = 'uli'

In [ ]:
def load_images(DIR, label):
    images = []
    labels = []
    for img_name in tqdm(os.listdir(DIR)):
        path = os.path.join(DIR, img_name)
        img = cv2.imread(path, cv2.IMREAD_COLOR)
        img = cv2.resize(img, (100, 100))
        img = img.astype('float32') / 255.0 
        
        images.append(img)
        labels.append(label)

        flipped_img = cv2.flip(img, 1)
        rotated_img = cv2.rotate(img, cv2.ROTATE_90_CLOCKWISE)
        zoomed_img = cv2.resize(img, None, fx=1.2, fy=1.2)
        zoomed_img = cv2.resize(zoomed_img, (100, 100))
        bright_img = cv2.convertScaleAbs(img, alpha=1.2, beta=30)
        
        images.extend([flipped_img, rotated_img, zoomed_img, bright_img])
        labels.extend([label, label, label, label])
    
    return images, labels

def extract_color_histogram(image):
    chans = cv2.split(image)
    features = []
    for chan in chans:
        hist = cv2.calcHist([chan], [0], None, [256], [0, 256])
        hist = cv2.normalize(hist, hist).flatten()
        features.extend(hist)
    return np.array(features)

def extract_texture(image):
    gray = cv2.cvtColor((image * 255).astype('uint8'), cv2.COLOR_BGR2GRAY)
    lbp = local_binary_pattern(gray, 24, 8, method="uniform")
    (hist, _) = np.histogram(lbp.ravel(), bins=np.arange(0, 24 + 3), range=(0, 24 + 2))
    hist = hist.astype("float")
    hist /= (hist.sum() + 1e-6)
    return hist

def extract_shape(image):
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    moments = cv2.moments(gray)
    hu_moments = cv2.HuMoments(moments).flatten()
    return hu_moments

def extract_edge(image):
    gray = cv2.cvtColor((image * 255).astype('uint8'), cv2.COLOR_BGR2GRAY)
    prewitt_x = cv2.filter2D(gray, -1, np.array([[1, 0, -1], [1, 0, -1], [1, 0, -1]]))
    prewitt_y = cv2.filter2D(gray, -1, np.array([[1, 1, 1], [0, 0, 0], [-1, -1, -1]]))
    edges = np.sqrt(prewitt_x**2 + prewitt_y**2)
    hist = cv2.calcHist([edges.astype('uint8')], [0], None, [256], [0, 256])
    hist = cv2.normalize(hist, hist).flatten()
    return hist

def extract_threshold(image):
    gray = cv2.cvtColor((image * 255).astype('uint8'), cv2.COLOR_BGR2GRAY)
    _, thresh = cv2.threshold(gray, 127, 255, cv2.THRESH_BINARY)
    hist = cv2.calcHist([thresh], [0], None, [256], [0, 256])
    hist = cv2.normalize(hist, hist).flatten()
    return hist

def extract_morphology(image):
    gray = cv2.cvtColor((image * 255).astype('uint8'), cv2.COLOR_BGR2GRAY)
    kernel = np.ones((5, 5), np.uint8)
    erosion = cv2.erode(gray, kernel, iterations=1)
    dilation = cv2.dilate(gray, kernel, iterations=1)
    opening = cv2.morphologyEx(gray, cv2.MORPH_OPEN, kernel)
    closing = cv2.morphologyEx(gray, cv2.MORPH_CLOSE, kernel)
    
    features = []
    for morph in [erosion, dilation, opening, closing]:
        hist = cv2.calcHist([morph], [0], None, [256], [0, 256])
        hist = cv2.normalize(hist, hist).flatten()
        features.extend(hist)
    return np.array(features)

In [ ]:
def derajat(img, angle):
    max_val = np.max(img)
    temp = np.zeros([max_val + 1, max_val + 1])

    if angle == 0:
        for i in range(len(img)):
            for j in range(len(img[i]) - 1):
                temp[img[i, j], img[i, j + 1]] += 1
    elif angle == 45:
        for i in range(len(img) - 1):
            for j in range(len(img[i]) - 1):
                temp[img[i + 1, j], img[i, j + 1]] += 1
    elif angle == 90:
        for i in range(len(img) - 1):
            for j in range(len(img[i]) - 1):
                temp[img[i + 1, j], img[i, j]] += 1
    elif angle == 135:
        for i in range(len(img) - 1):
            for j in range(len(img[i]) - 1):
                temp[img[i, j], img[i + 1, j + 1]] += 1

    data = temp + temp.T
    data /= np.sum(data)

    return data

def extract_glcm(image):
    gray = cv2.cvtColor((image * 255).astype('uint8'), cv2.COLOR_BGR2GRAY)
    angles = [0, 45, 90, 135]
    features = []
    for angle in angles:
        data = derajat(gray, angle)
        features.extend([data.mean(), data.var()])
    return np.array(features)

def extract_features(images):
    features = []
    for img in images:
        color_features = extract_color_histogram(img)
        texture_features = extract_texture(img)
        shape_features = extract_shape(img)
        edge_features = extract_edge(img)
        threshold_features = extract_threshold(img)
        morphology_features = extract_morphology(img)
        glcm_features = extract_glcm(img)
        combined_features = np.hstack([color_features, texture_features, shape_features, edge_features, threshold_features, morphology_features, glcm_features])
        features.append(combined_features)
    return np.array(features)

In [ ]:
X_ambon, y_ambon = load_images(AMBON_DIR, 'Ambon')
X_barangan, y_barangan = load_images(BARANGAN_DIR, 'Barangan')
X_cavendish, y_cavendish = load_images(CAVENDISH_DIR, 'Cavendish')
X_genderuwo, y_genderuwo = load_images(GENDERUWO_DIR, 'Genderuwo')
X_javaicecream, y_javaicecream = load_images(JAVAICECREAM_DIR, 'Javaicecream')
X_kepok, y_kepok = load_images(KEPOK_DIR, 'Kepok')
X_mas, y_mas = load_images(MAS_DIR, 'Mas')
X_nangka, y_nangka = load_images(NANGKA_DIR, 'Nangka')
X_raja, y_raja = load_images(RAJA_DIR, 'Raja')
X_susu, y_susu = load_images(SUSU_DIR, 'Susu')
X_tanduk, y_tanduk = load_images(TANDUK_DIR, 'Tanduk')
X_uli, y_uli = load_images(ULI_DIR, 'Uli')

X = X_ambon + X_barangan + X_cavendish + X_genderuwo + X_javaicecream + X_kepok + X_mas + X_nangka + X_raja + X_susu + X_tanduk + X_uli
y = y_ambon + y_barangan + y_cavendish + y_genderuwo + y_javaicecream + y_kepok + y_mas + y_nangka + y_raja + y_susu + y_tanduk + y_uli

X = np.array(X)
y = np.array(y)

X_features = extract_features(X)

df = pd.DataFrame(X_features)
df['label'] = y
df.to_csv('result/preprocessing.csv', index=False)

np.save('original_images.npy', X)